In [12]:
# pretrain down
# !wget http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
!wget http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r101_fpn_2x_coco/faster_rcnn_r101_fpn_2x_coco_bbox_mAP-0.398_20200504_210455-1d2dac9c.pth

--2021-05-14 03:08:49--  http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r101_fpn_2x_coco/faster_rcnn_r101_fpn_2x_coco_bbox_mAP-0.398_20200504_210455-1d2dac9c.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.25
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.25|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243518604 (232M) [application/octet-stream]
Saving to: ‘faster_rcnn_r101_fpn_2x_coco_bbox_mAP-0.398_20200504_210455-1d2dac9c.pth’

faster_rcnn_r101_fp 100%[===================>] 232.24M  9.33MB/s    in 30s     

2021-05-14 03:09:19 (7.82 MB/s) - ‘faster_rcnn_r101_fpn_2x_coco_bbox_mAP-0.398_20200504_210455-1d2dac9c.pth’ saved [243518604/243518604]



In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
# config file 들고오기
cfg = Config.fromfile('./configs/custom_augmentation/faster_rcnn_mosaic.py')

# load
cfg.load_from = 'faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'

print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(typ

In [3]:
model = build_detector(cfg.model)

2021-05-19 01:20:02,017 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-19 01:20:02,018 - mmdet - INFO - Use load_from_torchvision loader
2021-05-19 01:20:02,284 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [5]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.39s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-19 01:20:14,247 - mmdet - INFO - load checkpoint from faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2021-05-19 01:20:14,263 - mmdet - INFO - Use load_from_local loader


Done (t=1.26s)
creating index...
index created!


2021-05-19 01:20:14,726 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([44]).
2021-05-19 01:20:14,745 - mmdet - INFO - Start running, host: root@cca5f118b8f3, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/faster_rcnn_2x_batch8_wontrans_s50c65_num3
2021-05-19 01:20:14,746 - mmdet - INFO - workflow: [('train',

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 12.3 task/s, elapsed: 53s, ETA:     0s

...

2021-05-19 02:36:32,292 - mmdet - INFO - Epoch [10][40/330]	lr: 2.000e-02, eta: 1:42:59, time: 1.406, data_time: 0.075, memory: 8338, loss_rpn_cls: 0.0390, loss_rpn_bbox: 0.0470, loss_cls: 0.2916, acc: 90.9106, loss_bbox: 0.2548, loss: 0.6325, grad_norm: 2.2361
2021-05-19 02:37:17,203 - mmdet - INFO - Epoch [10][80/330]	lr: 2.000e-02, eta: 1:42:00, time: 1.123, data_time: 0.016, memory: 8338, loss_rpn_cls: 0.0359, loss_rpn_bbox: 0.0475, loss_cls: 0.2814, acc: 91.3593, loss_bbox: 0.2647, loss: 0.6294, grad_norm: 2.1362
2021-05-19 02:37:58,501 - mmdet - INFO - Epoch [10][120/330]	lr: 2.000e-02, eta: 1:40:55, time: 1.032, data_time: 0.016, memory: 8338, loss_rpn_cls: 0.0369, loss_rpn_bbox: 0.0421, loss_cls: 0.2793, acc: 91.1542, loss_bbox: 0.2475, loss: 0.6059, grad_norm: 2.1570
2021-05-19 02:38:41,426 - mmdet - INFO - Epoch [10][160/330]	lr: 2.000e-02, eta: 1:39:54, time: 1.073, data_time: 0.016, memory: 8338, loss_rpn_cls: 0.0313, loss_rpn_bbox: 0.0370, loss_cls: 0.2786, acc: 91.6071, l

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 9.1 task/s, elapsed: 72s, ETA:     0s

2021-05-19 02:43:43,707 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.56s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.54s).
Accumulating evaluation results...
DONE (t=0.86s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.193
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.306
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.198
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.050
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.188
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.390
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.376
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.376
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.376
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]

2021-05-19 02:44:47,376 - mmdet - INFO - Epoch [11][40/330]	lr: 2.000e-02, eta: 1:35:43, time: 1.416, data_time: 0.075, memory: 8338, loss_rpn_cls: 0.0304, loss_rpn_bbox: 0.0402, loss_cls: 0.2636, acc: 91.6034, loss_bbox: 0.2357, loss: 0.5699, grad_norm: 2.0574
2021-05-19 02:45:37,990 - mmdet - INFO - Epoch [11][80/330]	lr: 2.000e-02, eta: 1:34:54, time: 1.265, data_time: 0.017, memory: 8338, loss_rpn_cls: 0.0335, loss_rpn_bbox: 0.0419, loss_cls: 0.2728, acc: 91.5186, loss_bbox: 0.2452, loss: 0.5934, grad_norm: 2.1575
2021-05-19 02:46:19,776 - mmdet - INFO - Epoch [11][120/330]	lr: 2.000e-02, eta: 1:33:53, time: 1.045, data_time: 0.018, memory: 8338, loss_rpn_cls: 0.0361, loss_rpn_bbox: 0.0471, loss_cls: 0.2815, acc: 91.3513, loss_bbox: 0.2544, loss: 0.6191, grad_norm: 2.1020
2021-05-19 02:46:57,642 - mmdet - INFO - Epoch [11][160/330]	lr: 2.000e-02, eta: 1:32:47, time: 0.947, data_time: 0.017, memory: 8338, loss_rpn_cls: 0.0281, loss_rpn_bbox: 0.0369, loss_cls: 0.2543, acc: 92.1252, l